參考

In [1]:
# https://hackmd.io/@vagus/SyaRldHRv

### 這包會安裝一陣子 檔案內容較大

In [2]:
# !pip install pycaret[full]

In [3]:
# pip install pycaret[full]

如果發生錯誤 如
ERROR: Could not install packages due to an OSError: [WinError 5] 存取被拒。: 'C:\\Users\\User\\anaconda3\\envs\\python39\\Lib\\site-packages\\~andas\\_libs\\algos.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.
可以嘗試

In [4]:
# !pip3 install --upgrade tensorflow-gpu --user

In [5]:
# pip3 install --upgrade tensorflow-gpu --user

### 引入pycaret的內部資料

In [6]:
# !pip install pycaret

In [7]:
from pycaret.datasets import get_data
import pandas as pd
dataset = get_data('credit')
# dataset

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,90000,2,2,2,34,0,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
2,50000,2,2,1,37,0,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
3,50000,1,2,1,57,-1,0,-1,0,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
4,50000,1,1,2,37,0,0,0,0,0,...,19394.0,19619.0,20024.0,2500.0,1815.0,657.0,1000.0,1000.0,800.0,0


將5％的資料屏蔽，當作最後測試的資料

In [8]:
data = dataset.sample(frac=0.95,random_state=1)
data_useen =dataset.drop(data.index)
data.reset_index(inplace=True,drop=True)
print('Data for Modeling: '+str(data.shape))
print('Useen Data For Predictions: '+str(data_useen.shape))

Data for Modeling: (22800, 24)
Useen Data For Predictions: (1200, 24)


In [9]:
# 準備前處理的參數

In [10]:
from pycaret.classification import *

In [11]:
exp_clf01 = setup(data=data,target = 'default',session_id=1)

,Description,Value
0,Session id,1
1,Target,default
2,Target type,Binary
3,Original data shape,"(22800, 24)"
4,Transformed data shape,"(22800, 24)"
5,Transformed train set shape,"(15959, 24)"
6,Transformed test set shape,"(6841, 24)"
7,Numeric features,23
8,Preprocess,True
9,Imputation type,simple


In [12]:
# 選擇要建立的演算法

In [13]:
cart = create_model('dt')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7086,0.5952,0.3915,0.3582,0.3742,0.1848,0.1851
1,0.7206,0.5968,0.3746,0.3725,0.3736,0.1938,0.1938
2,0.7080,0.6001,0.4056,0.3609,0.3820,0.1917,0.1922
3,0.7143,0.5868,0.3577,0.3577,0.3577,0.1740,0.1740
4,0.7406,0.6326,0.4394,0.4205,0.4298,0.2620,0.2621
5,0.7187,0.6022,0.3859,0.3723,0.3790,0.1972,0.1973
6,0.7187,0.6020,0.3887,0.3730,0.3807,0.1988,0.1989
7,0.7362,0.6252,0.4254,0.4103,0.4177,0.2473,0.2473
8,0.7293,0.6243,0.4366,0.4005,0.4178,0.2419,0.2423


In [14]:
print(cart)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       random_state=1, splitter='best')


In [15]:
tuned_cart = tune_model(cart)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8202,0.7609,0.3634,0.6789,0.4734,0.3767,0.4035
1,0.8271,0.7626,0.3944,0.6965,0.5036,0.4085,0.4327
2,0.8239,0.7498,0.3859,0.6850,0.4937,0.3970,0.4210
3,0.8064,0.6796,0.2901,0.6438,0.4000,0.3038,0.3382
4,0.8340,0.7628,0.4225,0.7143,0.5310,0.4381,0.4604
5,0.8133,0.7284,0.3577,0.6447,0.4601,0.3583,0.3810
6,0.8258,0.7603,0.3521,0.7225,0.4735,0.3836,0.4193
7,0.8108,0.7553,0.3521,0.6345,0.4529,0.3496,0.3718
8,0.8189,0.7541,0.3549,0.6774,0.4658,0.3693,0.3974


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [16]:
print(tuned_cart)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=4, max_features=1.0, max_leaf_nodes=None,
                       min_impurity_decrease=0.001, min_samples_leaf=5,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       random_state=1, splitter='best')


In [17]:
# 可以直接預測資料，pycaret在前處理時會自動將資料分成70%訓練資料跟30%測試資料

In [18]:
predict_model(tuned_cart)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Decision Tree Classifier,0.8171,0.7420,0.3629,0.6619,0.4688,0.3695,0.3939


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default,prediction_label,prediction_score
10033,210000,2,1,2,26,1,2,2,0,0,...,125123.0,4500.0,0.0,4302.0,4455.0,4523.0,4516.0,0,0,0.6214
19121,150000,1,1,2,37,-1,-1,0,-1,0,...,82645.0,89524.0,19406.0,102989.0,2608.0,7845.0,0.0,0,0,0.8940
19488,80000,1,1,2,35,-1,-1,-1,-1,-1,...,19942.0,3983.0,6853.0,1719.0,1378.0,19942.0,2418.0,0,0,0.8940
20000,250000,2,1,1,40,-1,-1,-1,0,-1,...,326.0,326.0,652.0,0.0,326.0,326.0,326.0,1,0,0.7352
13005,360000,1,1,1,49,-1,-1,-2,-2,-1,...,2500.0,0.0,0.0,200.0,5084.0,0.0,0.0,1,0,0.8146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19394,300000,1,3,1,56,-2,-2,-2,-2,-2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0.7352
5048,280000,2,1,2,26,0,0,0,0,0,...,36334.0,1700.0,1600.0,1300.0,1500.0,2000.0,99334.0,0,0,0.8940
14895,50000,2,2,1,36,0,0,0,0,0,...,8738.0,2174.0,1785.0,296.0,306.0,444.0,10686.0,0,0,0.8146
11469,180000,2,1,2,29,-2,-2,-2,-2,-2,...,0.0,0.0,432.0,0.0,0.0,0.0,0.0,0,0,0.7352


In [21]:
from pycaret.utils import check_metric
check_metric(unseen_predictions['default'],unseen_predictions['Label'],metric='Accuracy')

ImportError: cannot import name 'check_metric' from 'pycaret.utils' (C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pycaret\utils\__init__.py)